In [95]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
PI = 3.1415926


#load image
def load_image(path):
    img = cv2.imread(path, 0)
    return img

#gamma to reduce noise
def gamma(img):
    height, width = img.shape
    for i in range(0, height, 1):
        for j in range(0, width, 1):
            img[i,j] = img[i,j] ** 0.5
            
#calculate gradient
def sobel_gradient(img):
    height, width = img.shape
    gx = cv2.Sobel(img, cv2.CV_16S, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_16S, 0, 1)
    grad_array = np.zeros((height, width, 2))
    for i in range(0, height, 1):
        for j in range(0, width, 1):
            grad_array[i,j,0] = math.sqrt(gx[i,j]**2 + gy[i,j]**2)
            if gy[i,j] == 0:
                grad_array[i,j,1] = 0
            else:
                grad_array[i,j,1] = np.arctan(gx[i,j]/gy[i,j])
    return grad_array

#get cell info
def get_single_cell(cell_array):
    cell_hist = [0] * 9
    height, width, channel = cell_array.shape
    for i in range(0, height, 1):
        for j in range(0, width, 1):
            ang = (PI + cell_array[i,j,1]) if cell_array[i,j,1] < 0 else cell_array[i,j,1]
            loop = 1
            while(ang > loop * PI / 9):
                loop += 1
            cell_hist[loop - 1] += cell_array[i,j,0]
    return cell_hist            


#devide gradient array into cells
def get_cells_info(grad_array, cell_size):
    height, width, channel = grad_array.shape
    cell_h, cell_w = cell_size
    cells_x_y_hist = np.zeros((height/cell_h, width/cell_w, 9))
    cells_x_y_grad = np.zeros((height/cell_h, width/cell_w, cell_h, cell_w, channel))
    for i in range(0, height/cell_h, 1):
        for j in range(0, width/cell_w, 1):
            cell = grad_array[i*cell_h: (i+1)*cell_h, j*cell_w: (j+1)*cell_w, :]
            cells_x_y_grad[i,j] = cell
            cells_x_y_hist[i,j] = get_single_cell(cell)
    return cells_x_y_grad, cells_x_y_hist

#tri linner interpolation, unfinished
def tri_linear_interpolation(cells_grad, cells_hist, block_size, stride):
    def four_points_weight(p, ptl, ptr, pbl, pbr):
        dis_tl = math.sqrt(sum([e * e for e in np.array(p) - np.array(ptl)]))
        dis_tr = math.sqrt(sum([e * e for e in np.array(p) - np.array(ptr)]))
        dis_bl = math.sqrt(sum([e * e for e in np.array(p) - np.array(pbl)]))
        dis_br = math.sqrt(sum([e * e for e in np.array(p) - np.array(pbr)]))
        s = dis_tl + dis_tr + dis_bl + dis_br
        return dis_br/s, dis_bl/s, dis_tr/s, dis_tl/s 
    def angle_weight(angle):
        loop = 1
        ang = (PI + angle) if angle < 0 else angle
        while(ang > loop * PI / 9):
            loop += 1
        ang
        return loop-1,  loop - ang/(PI/9)
    
    height, width, cell_h, cell_w, channel = cells_grad.shape
    block_h, block_w = block_size
    block_h_count, block_w_count = (height-block_h)/stride + 1, (width-block_w)/stride + 1
    for i in range(0, (height-block_h)/stride + 1, 1):
        for j in range(0, (width-block_w)/stride + 1, 1):
            tl_cell = cells_grad[i,j]
            tl_center = np.array(((i+0.5)*(cell_h-1), (j+0.5)*(cell_w-1)))
            tr_cell = cells_grad[i,j+1]
            tr_center = np.array(((i+0.5)*(cell_h-1), (j+1.5)*(cell_w-1)))
            bl_cell = cells_grad[i+1,j]
            bl_center = np.array(((i+1.5)*(cell_h-1), (j+0.5)*(cell_w-1)))
            br_cell = cells_grad[i+1, j+1]
            br_center = np.array(((i+1.5)*(cell_h-1), (j+1.5)*(cell_w-1)))
            for m in range(0, cell_h/2, 1):
                for n in range(0, cell_w/2, 1):
                    point_grad = tl_cell[m, n]
                    p_point = np.array((m+i*cell_h, n+j*cell_w))
                    p_center_tl = tl_center-cell_w
                    p_center_tr = tl_center-np.array((cell_h, 0))
                    p_center_bl = tl_center-np.array((0, cell_w))
                    p_center_br = tl_center                      
                    a,b,c,br_rate = four_points_weight(p_point, p_center_tl, p_center_tr, p_center_bl, p_center_br)
                    pos1, weight1 = angle_weight(point_grad[1])
                    #print pos1, weight1
                    cells_hist[i,j,pos1] += weight1 * br_rate * point_grad[0]
                    cells_hist[i,j,pos1+1] += (1-weight1) * br_rate * point_grad[0]
                    
                    
            
#assemble cells to block
def assemble_cells_to_block(cells_hist, block_size, stride):
    height, width, channel = cells_hist.shape
    block_h, block_w = block_size
    block_h_count, block_w_count = (height-block_h)/stride + 1, (width-block_w)/stride + 1
    block_hists = np.zeros((block_h_count, block_w_count, channel * block_h * block_w))
    for i in range(0, (height-block_h)/stride + 1, 1):
        for j in range(0, (width-block_w)/stride + 1, 1):
            single_block_his = []
            for m in range(0, block_h, 1):
                for n in range(0, block_w, 1):
                    single_block_his += list(cells_hist[i+m, j+n])
            block_hists[i,j] = single_block_his
    return block_hists
    
    
    
if __name__ == "__main__":
    #image size, 128*64
    img = load_image("G:/local/target_detect/target.jpg")
    gamma(img)
    grad_array = sobel_gradient(img)
    cell_size = (8,8)
    cells_grad, cells_hist = get_cells_info(grad_array, cell_size)
    block_size = (2,2)
    stride = 1
    tri_linear_interpolation(cells_grad, cells_hist, block_size, stride)
    block_hist = assemble_cells_to_block(cells_hist, block_size, stride)

    #print block_hist.shape
    #plt.plot(block_hist[4,5], 'b-')
    #plt.show()  